# ML regression with L1

- Under construction

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
#from sklearn import preprocessing

In [2]:
df=pd.read_csv("input/dataset_final_processed.csv")
df.head()

,Unnamed: 0,date,protestas_x,extremismo_x,rebelion_x,refugiados_x,precio_petroleo_x,juicio_x,corrupcion_x,inestabilidad_politica_x,...,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom,unemployment
0,3,2019-01-27,-0.986667,-29.993333,-21.453333,-6.425000,0.00,-111.880000,-92.164000,-7.770000,...,2.0,56.0,27.0,10.0,2.0,2.0,35.0,50.0,5.0,21.0
1,4,2019-02-03,-16.754286,-2.260000,-17.413333,-15.266667,0.00,-485.394286,-168.828571,-23.500000,...,1.0,51.0,35.0,15.0,3.0,1.0,27.0,44.0,4.0,23.0
2,5,2019-02-10,-98.830000,-5.644000,-26.305000,0.000000,3.72,-232.251429,-106.468571,-36.120000,...,2.0,50.0,100.0,9.0,3.0,2.0,16.0,41.0,3.0,23.0
3,6,2019-02-17,-41.448571,-4.226667,-26.570000,1.276000,0.00,-233.122857,-106.342857,-24.146667,...,2.0,52.0,78.0,11.0,3.0,3.0,11.0,49.0,4.0,23.0
4,7,2019-02-24,-35.362857,0.000000,-28.106667,-20.310000,0.00,-173.451429,-112.497143,-5.160000,...,1.0,50.0,40.0,11.0,3.0,2.0,13.0,48.0,4.0,20.0


- I have all this columns to play with, and I'm going to infer the column "unemployment"

In [3]:
df.drop(columns='Unnamed: 0', inplace=True)

In [4]:
print("I have ", len(df.columns), " columns to play with")#columns=list(df.columns)

I have  132  columns to play with


In [5]:
df.tail()

,date,protestas_x,extremismo_x,rebelion_x,refugiados_x,precio_petroleo_x,juicio_x,corrupcion_x,inestabilidad_politica_x,terrorismo_x,...,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom,unemployment
89,2020-10-11,-7.955000,-4.300,-4.933333,-3.205,0.0,-44.481429,-53.120000,-7.48,-3.300000,...,100.0,88.0,45.0,75.0,17.0,7.0,8.0,47.0,22.0,30.0
90,2020-10-18,-10.885714,-4.375,-2.670000,-3.600,0.0,-57.642857,-28.765714,0.00,-7.586000,...,43.0,88.0,48.0,90.0,17.0,8.0,10.0,46.0,20.0,27.0
91,2020-10-25,-15.368000,-3.420,0.000000,-3.880,0.0,-73.910000,-18.664286,-39.07,-4.770000,...,24.0,82.0,46.0,91.0,10.0,5.0,8.0,45.0,20.0,0.0
92,2020-11-01,-6.241667,0.000,-1.250000,-4.330,0.0,-39.984286,-30.545714,-14.97,-3.846667,...,38.0,90.0,47.0,100.0,8.0,7.0,11.0,42.0,18.0,0.0
93,2020-11-08,-8.295000,0.000,-4.800000,-0.665,0.0,-103.931667,-36.138571,-3.99,-5.055000,...,25.0,82.0,47.0,99.0,13.0,7.0,11.0,39.0,21.0,0.0


# Normalise

- This dataset is composed by datasets with a concrete range, and other dataset with a different scale. Let's normalise it

- I will not play with date column, I think there's pointless for a linear regression with my range of time, and without NN, so I'll keep it out of calculations

- Later i will append the *date* column, for dashboarding

In [6]:
#to_normalise = df.drop(columns="date").values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
#normalised = min_max_scaler.fit_transform(to_normalise)
#df_normalised = pd.DataFrame(normalised)
#df_normalised.columns = list(df.drop(columns="date").columns)
#df_normalised.head()

In [7]:
# Transform into standard normal distribution using the z-score definition
df_normalized = df.drop(columns="date") #returns a numpy array
df_normalized.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))

,protestas_x,extremismo_x,rebelion_x,refugiados_x,precio_petroleo_x,juicio_x,corrupcion_x,inestabilidad_politica_x,terrorismo_x,vigilancia_x,...,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom,unemployment
0,0.079454,-0.227000,-0.048510,0.004997,0.055642,0.126841,-0.005161,0.042840,-0.408330,-0.129363,...,-0.125725,-0.244889,-0.186170,-0.394903,-0.090157,-0.069955,0.191150,0.040725,-0.098157,-0.082979
1,0.018951,0.038086,-0.013077,-0.073158,0.055642,-0.642666,-0.290909,-0.007129,0.077374,-0.537050,...,-0.135826,-0.342929,-0.083606,-0.339958,-0.080056,-0.080056,0.106043,-0.053025,-0.108467,-0.062979
2,-0.295988,0.005740,-0.091062,0.061790,0.324817,-0.121146,-0.058478,-0.047218,-0.842861,-0.335707,...,-0.125725,-0.362537,0.749727,-0.405892,-0.080056,-0.069955,-0.010978,-0.099900,-0.118776,-0.062979
3,-0.075805,0.019288,-0.093386,0.073069,0.055642,-0.122941,-0.058010,-0.009183,-0.077503,0.065125,...,-0.125725,-0.323321,0.467676,-0.383914,-0.080056,-0.059854,-0.064169,0.025100,-0.108467,-0.062979
4,-0.052453,0.059688,-0.106864,-0.117738,0.055642,-0.000007,-0.080948,0.051131,-0.011607,-0.014582,...,-0.135826,-0.362537,-0.019504,-0.383914,-0.080056,-0.069955,-0.042893,0.009475,-0.108467,-0.092979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,0.052715,0.018587,0.096379,0.033459,0.055642,0.265694,0.140365,0.043761,0.134421,0.184980,...,0.864174,0.382562,0.044599,0.319383,0.061358,-0.019450,-0.096084,-0.006150,0.077100,0.007021
90,0.041470,0.017870,0.116230,0.029968,0.055642,0.238579,0.231140,0.067523,0.104915,0.015622,...,0.288416,0.382562,0.083061,0.484218,0.061358,-0.009349,-0.074808,-0.021775,0.056482,-0.022979
91,0.024270,0.026998,0.139647,0.027493,0.055642,0.205066,0.268790,-0.056589,0.124301,0.133559,...,0.096497,0.264914,0.057420,0.495207,-0.009349,-0.039652,-0.096084,-0.037400,0.056482,-0.292979
92,0.059290,0.059688,0.128684,0.023515,0.055642,0.274959,0.224505,0.019968,0.130657,0.177205,...,0.237911,0.421777,0.070240,0.594108,-0.029551,-0.019450,-0.064169,-0.084275,0.035863,-0.292979


# Selecting best features using RFE

(Recursive Feature Elimination)

In [8]:
X=df_normalized.drop(columns="unemployment")
target=df_normalized["unemployment"]

In [33]:
target.head()

0    21.0
1    23.0
2    23.0
3    23.0
4    20.0
Name: unemployment, dtype: float64

In [10]:
rfc = LinearRegression()
rfecv = RFE(estimator=rfc, step=1, n_features_to_select=15)
rfecv.fit(X, target)

RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                               normalize=False),
    n_features_to_select=15, step=1, verbose=0)

In [11]:
rfecv.get_params()

{'estimator__copy_X': True,
 'estimator__fit_intercept': True,
 'estimator__n_jobs': None,
 'estimator__normalize': False,
 'estimator': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 'n_features_to_select': 15,
 'step': 1,
 'verbose': 0}

In [37]:
rfecv.score(X, target)

0.891056332393066

In [93]:
score = pd.DataFrame({"date": [max(df["date"])], 'score': [round(rfecv.score(X, target),4)]})
score.to_csv("input/weekly_score.csv")
score

,date,score
0,2020-11-08,0.8911


In [94]:
features=pd.DataFrame()
features["features"]=X.columns
features["top_important"]=rfecv.ranking_
features.sort_values(by=["top_important"], inplace=True)
features.to_csv("input/ranking_of_features.csv")
features.head()

,features,top_important
105,pandemia,1
114,refugiados,1
33,prosperidad_economica_y_finanzas,1
43,censura_en_medios,1
45,energias_renovables,1


In [95]:
result=pd.DataFrame()
result["date"]=df["date"]
result["real_searches"]=df["unemployment"]
result["infered_searches"]=pd.DataFrame(rfecv.predict(X))
result["infered_searches"]=result["infered_searches"].apply(lambda x: 0 if x<0 else round(x,2))
result

,date,real_searches,infered_searches
0,2019-01-27,21.0,19.04
1,2019-02-03,23.0,26.93
2,2019-02-10,23.0,16.01
3,2019-02-17,23.0,21.08
4,2019-02-24,20.0,20.08
...,...,...,...
89,2020-10-11,30.0,35.95
90,2020-10-18,27.0,29.38
91,2020-10-25,0.0,0.00
92,2020-11-01,0.0,16.91


- https://www.datacamp.com/community/tutorials/feature-selection-python
- https://github.com/albertovpd/machine_learning_specialization_washington_university/blob/master/course_tasks/c2_regression-w1_quizz_1-basic_regression_math.ipynb
- https://github.com/albertovpd/machine_learning_specialization_washington_university/blob/master/course_tasks/c2_regression-w2_quizz_1-basic_multiple_regression.ipynb
- https://github.com/albertovpd/machine_learning_specialization_washington_university/blob/master/course_tasks/c2_regression-w3_quizz_1-exploring_variance_bias_tradeoff.ipynb
- https://github.com/albertovpd/machine_learning_specialization_washington_university/blob/master/course_tasks/c2_regression-w4_quizz_1-rigde_regression_and_validation_k-folds_method.ipynb
- https://github.com/albertovpd/machine_learning_specialization_washington_university/blob/master/course_tasks/c2_regression-w5_quizz_1-using_lasso_to_select_features.ipynb
- https://github.com/albertovpd/machine_learning_specialization_washington_university/blob/master/course_tasks/c2_regression-w5_quizz_2-lasso_with_coordinate_descent.ipynb
